In [1]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [5]:
data_file = open('/content/intents.json').read()
intents = json.loads(data_file)

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
#chaque intent contient : tag , patterns ,responses ,context.
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [12]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
#sorted
classes = sorted(classes)

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('/content/words.pkl','wb'))
pickle.dump(classes,open('/content/classes.pkl','wb'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
588 documents
196 classes [['a15 battery'], ['a15 biometrics'], ['a15 box'], ['a15 camera'], ['a15 chips'], ['a15 connectivity'], ['a15 display'], ['a15 memory'], ['a15 network'], ['a15 position'], ['a15 sensors'], ['a15 size'], ['a15 system'], ['a15 video'], ['a15s battery'], ['a15s biometrics'], ['a15s box'], ['a15s camera'], ['a15s chips'], ['a15s connectivity'], ['a15s display'], ['a15s memory'], ['a15s network'], ['a15s position'], ['a15s sensors'], ['a15s size'], ['a15s system'], ['a15s video'], ['a74 battery'], ['a74 biometrics'], ['a74 box'], ['a74 camera'], ['a74 chips'], ['a74 connectivity'], ['a74 display'], ['a74 memory'], ['a74 network'], ['a74 position'], ['a74 sensors'], ['a74 size'], ['a74 system'], ['a74 video'], ['a94 battery'], ['a94 biometrics'], ['a94 box'], ['a94 camera'], ['a94 chips'], ['a94 connectivity'], ['a94 display'], ['a94 memory'], ['a94 net

In [13]:
#Our input will be the pattern and the output will be the class which our input pattern belongs to.
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
      bag.append(1) if w in pattern_words else bag.append(0)
      
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [14]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
118/118 [==============================] - 1s 1ms/step - loss: 5.2970 - accuracy: 0.0047
Epoch 2/200
118/118 [==============================] - 0s 1ms/step - loss: 5.2766 - accuracy: 0.0100
Epoch 3/200
118/118 [==============================] - 0s 1ms/step - loss: 5.2545 - accuracy: 0.0110
Epoch 4/200
118/118 [==============================] - 0s 1ms/step - loss: 5.2111 - accuracy: 0.0196
Epoch 5/200
118/118 [==============================] - 0s 1ms/step - loss: 5.1144 - accuracy: 0.0293
Epoch 6/200
118/118 [==============================] - 0s 1ms/step - loss: 5.0005 - accuracy: 0.0253
Epoch 7/200
118/118 [==============================] - 0s 1ms/step - loss: 4.7411 - accuracy: 0.0369
Epoch 8/200
118/118 [==============================] - 0s 1ms/step - loss: 4.5230 - accuracy: 0.0499
Epoch 9/200
118/118 [==============================] - 0s 1ms/step - loss: 4.2748 - accuracy: 0.0641
Epoch 10/200
118/118 [==============================] - 0s 1ms/step - loss: 4.1718 - accura

In [16]:
intents = json.loads(open('/content/intents.json').read())
words = pickle.load(open('/content/words.pkl','rb'))
classes = pickle.load(open('/content/classes.pkl','rb'))

In [17]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)

    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    return sentence_words

In [18]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [19]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [20]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [21]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res


In [23]:
#test
chatbot_response('caractéristiques de camera de a74 ?')

'2G: GSM, 850/900/1800/1900 MHz'